# Covid-19 Analysis & Prediction by Kushagra Srivastava

In [5]:
import os 
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from math import sqrt
from datetime import datetime

%matplotlib inline

Imporing Data Set

raw_data_confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
raw_data_deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
raw_data_Recovered = pd.read_csv('time_series_covid19_recovered_global.csv')


print("The Shape of confirmed is: ", raw_data_confirmed.shape)
print("The Shape of deaths is: ", raw_data_deaths.shape)
print("The Shape of recovered is: ", raw_data_Recovered.shape)

Un-Pivoting the data

In [9]:
raw_data_confirmed2 = pd.melt(raw_data_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                              var_name=['Date'])
raw_data_deaths2 = pd.melt(raw_data_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                           var_name=['Date'])
raw_data_Recovered2 = pd.melt(raw_data_Recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                              var_name=['Date'])

print("The Shape of Confirmed is: ", raw_data_confirmed2.shape)
print("The Shape of deaths is: ", raw_data_deaths2.shape)
print("The Shape of recovered is: ", raw_data_Recovered2.shape)

The Shape of Confirmed is:  (310386, 6)
The Shape of deaths is:  (310386, 6)
The Shape of recovered is:  (294276, 6)


Converting the new column to dates

In [17]:
raw_data_confirmed2['Date'] = pd.to_datetime(raw_data_confirmed2['Date'])
raw_data_deaths2['Date'] = pd.to_datetime(raw_data_deaths2['Date'])
raw_data_Recovered2['Date'] = pd.to_datetime(raw_data_Recovered2['Date'])

Renaming the Values

In [20]:
raw_data_confirmed2.columns = raw_data_confirmed2.columns.str.replace('value', 'Confirmed')
raw_data_deaths2.columns = raw_data_deaths2.columns.str.replace('value', 'Deaths')
raw_data_Recovered2.columns = raw_data_Recovered2.columns.str.replace('value', 'Recovered')

raw_data_confirmed2.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0


Finding the NULL values

In [21]:
raw_data_Recovered2.isnull().sum()

Province/State    213726
Country/Region         0
Lat                 1074
Long                1074
Date                   0
Recovered              0
dtype: int64

Dealing with NULL values

In [29]:
raw_data_confirmed2['Province/State'].fillna(raw_data_confirmed2['Country/Region'], inplace=True)
raw_data_deaths2['Province/State'].fillna(raw_data_deaths2['Country/Region'], inplace=True)
raw_data_Recovered2['Province/State'].fillna(raw_data_Recovered2['Country/Region'], inplace=True)

raw_data_Recovered2.isnull().sum()

Province/State       0
Country/Region       0
Lat               1074
Long              1074
Date                 0
Recovered            0
dtype: int64

In [30]:
print("The Shape of confirmed is: ", raw_data_confirmed2.shape)
print("The Shape of deaths is: ", raw_data_deaths2.shape)
print("The Shape of recovered is: ", raw_data_Recovered2.shape)

The Shape of confirmed is:  (310386, 6)
The Shape of deaths is:  (310386, 6)
The Shape of recovered is:  (294276, 6)


JOIN

In [31]:
full_join = raw_data_confirmed2.merge(raw_data_deaths2[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0


In [32]:
full_join = full_join.merge(raw_data_Recovered2[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0


Adding Month and Year as a new Column

In [33]:
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020


In [34]:
full_join2 = full_join.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)
full_join2.head()

,Province/State,Country/Region,Lat,Long,Date Minus 1,Confirmed - 1,Deaths - 1,Recovered - 1,Month-Year,Date - 1
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020,2020-01-23
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020,2020-01-23
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020,2020-01-23
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020,2020-01-23
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020,2020-01-23


In [35]:
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT


In [36]:
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']

print(full_join3.shape)

(310386, 17)


In [37]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


Breaking the numbers by Day

In [38]:
#creating a new df    
full_join2 = full_join.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

#minus_onedf.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1'}, inplace=True)

full_join3.head()

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']

print(full_join3.shape)

(310386, 17)


In [39]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


In [40]:
# Additing manually the numbers for first day

full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

# deleting columns
del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

C:\Users\Kushagra Srivastava\AppData\Local\Temp\ipykernel_1980\2712811409.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
C:\Users\Kushagra Srivastava\AppData\Local\Temp\ipykernel_1980\2712811409.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
C:\Users\Kushagra Srivastava\AppData\Local\Temp\ipykernel_1980\2712811409.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

Removing Negative Values

In [41]:
full_join3[full_join3["Deaths Daily"]<0]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
15753,Iceland,Iceland,64.963100,-19.020800,2020-03-16,180,0,0.0,Mar-2020,9.0,-5.0,-8.0
16030,Germany,Germany,51.165691,10.451526,2020-03-17,7156,12,67.0,Mar-2020,1144.0,-1.0,0.0
16689,Philippines,Philippines,12.879721,121.774017,2020-03-19,217,17,8.0,Mar-2020,15.0,-2.0,3.0
16909,Iceland,Iceland,64.963100,-19.020800,2020-03-20,409,0,5.0,Mar-2020,79.0,-1.0,0.0
17199,India,India,20.593684,78.962880,2020-03-21,330,4,23.0,Mar-2020,86.0,-1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
302672,Unknown,China,NaN,NaN,2022-12-04,1393647,0,0.0,Dec-2022,26896.0,-2.0,0.0
303912,Lithuania,Lithuania,55.169400,23.881300,2022-12-08,1279559,9440,0.0,Dec-2022,511.0,-2.0,0.0
305068,Lithuania,Lithuania,55.169400,23.881300,2022-12-12,1280625,9441,0.0,Dec-2022,51.0,-1.0,0.0
308090,Western Australia,Australia,-31.950500,115.860500,2022-12-23,1255595,837,0.0,Dec-2022,0.0,-4.0,0.0


In [45]:
full_join3['Deaths Daily']=np.where(full_join3['Deaths Daily']<0 ,0,full_join3['Deaths Daily'])
full_join3['Confirmed Daily']=np.where(full_join3['Confirmed Daily']<0 ,0,full_join3['Confirmed Daily'])
full_join3['Recovered Daily']=np.where(full_join3['Recovered Daily']<0 ,0,full_join3['Recovered Daily'])

full_join3

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
1,Albania,Albania,41.153300,20.168300,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
2,Algeria,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
3,Andorra,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
4,Angola,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
310381,West Bank and Gaza,West Bank and Gaza,31.952200,35.233200,2022-12-30,703228,5708,0.0,Dec-2022,0.0,0.0,0.0
310382,Winter Olympics 2022,Winter Olympics 2022,39.904200,116.407400,2022-12-30,535,0,0.0,Dec-2022,0.0,0.0,0.0
310383,Yemen,Yemen,15.552727,48.516388,2022-12-30,11945,2159,0.0,Dec-2022,0.0,0.0,0.0
310384,Zambia,Zambia,-13.133897,27.849332,2022-12-30,334425,4024,0.0,Dec-2022,131.0,1.0,0.0


Exporting Output File

In [49]:
path = "E:\\Projects\\Covid 19 Analysis\\"

os.chdir(path)

full_join3.to_csv('CoronaVirus Data.csv')